In [33]:
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm, tqdm_notebook
import math
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras_tqdm import TQDMNotebookCallback
from tensorboard.plugins.hparams import api as hp
from livelossplot.tf_keras import PlotLossesCallback

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave
from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from skimage.transform import resize
from timeit import default_timer as timer
from datetime import timedelta
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.models import Model, load_model
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import plot_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks, butter
from scipy.integrate import simps
from scipy.io import loadmat
from numpy import trapz

In [34]:
df = pd.read_csv('152-individual-result-r.csv')
df.shape

(864, 7)

In [35]:
df.head()

,sub,target,band,input,p,b,no_of_obs
0,sub02,mep_size_log,theta,power,0.038,-0.15199,127
1,sub02,mep_size_log,theta,phase,0.217,-0.03232,127
2,sub02,mep_size_log,theta,interaction,0.152,NaN,127
3,sub02,mep_size_log,mu,power,0.579,-0.01361,122
4,sub02,mep_size_log,mu,phase,0.416,0.04428,122


In [36]:
df2 = []

for sub in df['sub'].unique():
    dfsub = df[df['sub'] == sub]
    for feature in ['mep_size_log', 'mep_latency_log', 'mep_area_log', 'mep_duration_log']:
        dffeature = dfsub[dfsub['target'] == feature]
        for eegfeature in ['power', 'phase', 'interaction']:
            dfeeg = dffeature[dffeature['input'] == eegfeature]
            row = {'sub': sub, 'mep': feature, 'input': eegfeature}
            for band in ['theta', 'mu', 'beta', 'gamma']:
                dfband = dfeeg[dfeeg['band'] == band]
                row[band + '_p'] = dfband['p'].values[0]
                row[band + '_B'] = dfband['b'].values[0]
            df2.append(row)
df2 = pd.DataFrame(df2)

In [37]:
df2 = df2[['sub', 'mep', 'input', 'theta_p', 'theta_B', 'mu_p', 'mu_B', 'beta_p', 'beta_B', 'gamma_p', 'gamma_B']]
df2.to_excel('152-organized-r.xlsx')